In [2]:
# Libraries

from selenium import webdriver
from selenium.webdriver import Chrome
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from bs4 import BeautifulSoup
import pandas as pd
import time
import uuid
import re

In [3]:
# define classes of information to be collected

class Reviews:
    def __init__(self, review_id, item_id, review_time, review_variant, star_rating, review_content):
        self.review_id = review_id
        self.item_id=item_id
        self.review_time = review_time
        self.review_variant = review_variant
        self.star_rating  = star_rating
        self.review_content = review_content

class Items:
    def __init__(self, item_id, item_name, item_ratings, ratings_num, item_sold, shop_name, item_price, item_url, item_desc, image_url):
        self.item_id=item_id
        self.item_name=item_name
        self.item_ratings=item_ratings
        self.ratings_num=ratings_num
        self.item_sold=item_sold
        self.shop_name=shop_name
        self.item_price=item_price
        self.item_url=item_url
        self.item_desc=item_desc
        self.image_url=image_url


In [4]:
#Create Chrome options
chrome_options= Options()
chrome_options.add_argument('--no sandbox')
chrome_options.add_argument('disable-notifications')
chrome_options.add_argument('disable-inforbars')

# Set the path to the Chrome driver executable
chrome_driver_path = 'D:\Chrome web driver\chromedriver-win64\chromedriver.exe'

# Create an instance of the Chrome browser
browser = webdriver.Chrome(executable_path=chrome_driver_path, options=chrome_options)

# Navigate to a URL 
browser.get("https://shopee.com.my/")

C:\Users\Asus\AppData\Local\Temp\ipykernel_5436\1499064468.py:11: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  browser = webdriver.Chrome(executable_path=chrome_driver_path, options=chrome_options)


In [17]:
soup=BeautifulSoup(browser.page_source, "html.parser")

the_links=[]

parent_div = soup.findAll('div', class_="col-xs-2-4 shopee-search-item-result__item")

for parent_div in parent_div:
    a=parent_div.find('a')
    if (a!=None):
        href=a.get('href')
        the_links.append(href)

        # print the links
        print(href)
        

/【Ready-Stock】Crocs-ClassicClog-Men-Slippers-Official-Authentic-Casual-Women's-Slippers-Beach-Shoes-Flat-Outdoor-Slipper-i.299805592.14626881466?sp_atk=1ff9ba14-5316-4bd7-9ceb-a9dfb21ddfac&xptdk=1ff9ba14-5316-4bd7-9ceb-a9dfb21ddfac
/READY-STOCK-Xiaoyulu-TAOMI-Kasut-Jururawat-Ringan-Misi-Hitam-Putih-Woman-Cut-outs-Wedge-Sneakers-Nurse-Shoes-Swing-Work-Single-Shoes-i.68738438.1140855204?sp_atk=5b0802de-636e-4d79-bee7-008fadee5000&xptdk=5b0802de-636e-4d79-bee7-008fadee5000
/【Malaysia-Ready-Stock】Classic-street-shoes-Sepatu-Skateboarding-kekasih-kasut-sukan-kekasih-kasut-sukan-untuk-lelaki-dan-i.460458715.23227008418?sp_atk=205f7393-b6cd-44a1-b031-40b7c0eb49de&xptdk=205f7393-b6cd-44a1-b031-40b7c0eb49de
/READY-STOCK💝WEBEE-600-Kelsea-Women's-High-Heels-Shoes-Kasut-Tinggi-Wanita-Heel-Transparent-i.7940214.10305233675?sp_atk=d34274a4-5e5a-4222-88ae-90020ff21806&xptdk=d34274a4-5e5a-4222-88ae-90020ff21806
/Slipper-selipar-unisex-indoor-outdoor-Japanese-Comfortable-Thick-Sole-Shower-Home-Slippers

In [18]:
len(the_links)

60

In [19]:
items_list=[]
reviews_list=[]

In [20]:
for i in range (0,len(the_links)):
    
    # open link
    links='https://shopee.com.my'+ the_links[i]
    browser.get(links)
    time.sleep(10)

    # parse html using beautifulsoup
    soup=BeautifulSoup(browser.page_source, "html.parser")

    # extract product information
    id=uuid.uuid4()
    title = soup.find('div', attrs={'class':'_44qnta'}).find('span').get_text(strip=True)
    ratings=soup.find('div', class_="_1k47d8 _046PXf").get_text(strip=True)
    ratings_num=soup.select('div._1k47d8:not(div._1k47d8._046PXf)')[0].get_text(strip=True)
    sold=soup.find('div', class_="e9sAa2").get_text(strip=True)
    shop_name=soup.find('div',class_="VlDReK").get_text(strip=True)
    price = soup.find('div', class_='pqTWkA').get_text(strip=True)

    # extract description
    descriptions = soup.find_all('p', class_='irIKAp')
    

    desc_array=[]
    for description in descriptions:
        desc = re.sub(r'<[^>]+>', '', str(description))  # Remove HTML tags
        desc = desc.strip()  # Remove leading/trailing whitespaces
        desc_array.append(desc)

    text = "\n".join(desc_array)

    # extract picture url
    img_tag= soup.find('img', class_='_7D4JtJ')

    if img_tag:
        image = img_tag['src']
    else:
        image=None

    # append item to a list
    items_list.append(Items(id,title,ratings,ratings_num,sold,shop_name, price, links, description, image))

    # reviews extraction
    elements = browser.find_elements(By.CLASS_NAME,"product-rating-overview__filter")
    counter=5
    for j in range(1,6):
        # select star categories
        elements[j].click()
        
        # loop is for each category
        for k in range (0,31): 
            soup=BeautifulSoup(browser.page_source, "html.parser")
            review_containers=soup.findAll('div',class_="shopee-product-rating__main")

            no_reviews_found = False

            for rc in review_containers:
                review_id=uuid.uuid4()
                item_id=id
                time_var=rc.find('div',class_="shopee-product-rating__time").get_text(strip=True).split("|")

                if(len(time_var)==2):
                    review_time=time_var[0]
                    variant=time_var[1]
                else:
                    review_time=time_var[0]
                    variant="None"

                star_rating=counter
                reviews=rc.find('div',class_="Rk6V+3")

                if(reviews!=None):
                    reviews=reviews.get_text(strip=True)
                else:
                    reviews="None"
                    no_reviews_found = True

                reviews_list.append(Reviews(review_id, item_id, review_time,variant,star_rating,reviews))
            
            # no more reviews break
            if no_reviews_found:
                print("No reviews found, breaking the loop")
                break

            time.sleep(1)
            try:
                WebDriverWait(browser, 1).until(EC.element_to_be_clickable((By.CSS_SELECTOR, "button[class^='shopee-icon-button shopee-icon-button--right "))).click()
            except:
                print("not found")
                break
            time.sleep(1)
        
        counter=counter-1
        time.sleep(2)
    print(i) 
    
    

No reviews found, breaking the loop
No reviews found, breaking the loop
No reviews found, breaking the loop
No reviews found, breaking the loop
No reviews found, breaking the loop
0
No reviews found, breaking the loop
No reviews found, breaking the loop
No reviews found, breaking the loop
No reviews found, breaking the loop
No reviews found, breaking the loop
1
No reviews found, breaking the loop
No reviews found, breaking the loop
No reviews found, breaking the loop
No reviews found, breaking the loop
No reviews found, breaking the loop
2
No reviews found, breaking the loop
No reviews found, breaking the loop
No reviews found, breaking the loop
No reviews found, breaking the loop
No reviews found, breaking the loop
3
No reviews found, breaking the loop
No reviews found, breaking the loop
No reviews found, breaking the loop
No reviews found, breaking the loop
No reviews found, breaking the loop
4
No reviews found, breaking the loop
No reviews found, breaking the loop
No reviews found, 

In [21]:
items_df = pd.DataFrame([vars(obj) for obj in items_list])
reviews_df = pd.DataFrame([vars(obj) for obj in reviews_list])

In [22]:
items_df.shape

(60, 10)

In [23]:
items_df.head()

,item_id,item_name,item_ratings,ratings_num,item_sold,shop_name,item_price,item_url,item_desc,image_url
0,248c7491-982d-4b7f-86d5-4d989a1e714f,【Ready Stock】Crocs ClassicClog Men Slippers O...,4.9,1.9k,5.5k,C r o c s Literide Store,RM7.99 - RM32.59,https://shopee.com.my/【Ready-Stock】Crocs-Class...,[❤️❤️Dear customer❤️❤️\nSince we are a new sto...,https://down-my.img.susercontent.com/file/5189...
1,08499ad7-40f4-4cb1-a169-9bfa4a4ce280,READY STOCK Xiaoyulu TAOMI Kasut Jururawat Ri...,4.9,14.1k,31.7k,Xiaoyulu Shoes Wholesale,RM27.50 - RM31.90,https://shopee.com.my/READY-STOCK-Xiaoyulu-TAO...,[📱Follow Xiaoyulu ❤ ( ◠‿◠ )❤\n\nIT is Pull whi...,https://down-my.img.susercontent.com/file/7fb1...
2,c85fd3f1-b66e-458d-98c6-60cf51644f07,【Malaysia Ready Stock】Classic street shoes Sep...,4.9,599,1.4k,DFG Shoes,RM36.00 - RM42.00,https://shopee.com.my/【Malaysia-Ready-Stock】Cl...,[**STOK SEDIA ADA di Malaysia**\npenghantaran ...,https://down-my.img.susercontent.com/file/63af...
3,93537540-117a-4f68-8d86-9d7efc018ea3,READY STOCK💝WEBEE 600 Kelsea Women's High Heel...,4.8,1.9k,5.2k,WEBEE,RM10.00 - RM16.80,https://shopee.com.my/READY-STOCK💝WEBEE-600-Ke...,[📱Follow WEBEE ❤ ( ◠‿◠ )❤\n🎒READY STOCK IN MA...,https://down-my.img.susercontent.com/file/8db3...
4,0330dad5-d990-4770-9fe0-f9cc2790c404,Slipper selipar unisex indoor & outdoor Japane...,4.6,11.2k,46.7k,258 wholesale mall,RM3.79 - RM4.50,https://shopee.com.my/Slipper-selipar-unisex-i...,[Anti-Slip Home Slipper\n\nDesign: Premium(*Vi...,https://down-my.img.susercontent.com/file/53b5...


In [24]:
reviews_df.shape

(5122, 6)

In [25]:
reviews_df.tail()

,review_id,item_id,review_time,review_variant,star_rating,review_content
5117,ddc72221-b013-4311-a871-d60cd0a08f2e,01bc0b81-a055-4bf2-89d9-e558488717e4,2023-04-19 12:45,"Variation: Black,42",2,"Order saiz 42 tapi sampai yang kecil,tali longgar"
5118,565c706b-69f7-4614-a0b1-0c68f2bb66ec,01bc0b81-a055-4bf2-89d9-e558488717e4,2023-01-18 18:33,"Variation: Black,40",2,None
5119,6b7bdba8-a913-48cd-b9c8-2ae28166e57c,01bc0b81-a055-4bf2-89d9-e558488717e4,2023-06-25 23:53,"Variation: White,41",1,"Kaler tulis putih,sampai2 kaler krim.penghanta..."
5120,f4c4446e-4226-49d4-8280-41c4dba3a5be,01bc0b81-a055-4bf2-89d9-e558488717e4,2023-06-28 16:54,"Variation: Black,35",1,Comfortability:not comfortable at allBad quali...
5121,a9fc5a25-1245-44cc-94ab-ebe6cde27c4a,01bc0b81-a055-4bf2-89d9-e558488717e4,2023-06-25 11:56,"Variation: White,40",1,None


In [26]:
reviews_df.to_json('womenshoes_reviews_2.json', orient='records', lines=True, force_ascii=False, default_handler=str)
items_df.to_json('womenshoes_items_2.json', orient='records', lines=True, force_ascii=False, default_handler=str)